Pulsed ODMR with RF

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegenerator')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')  

def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return    

In [3]:
mw_freq = 1600.0e+6 # in MHz
mw_power = -32

# Set new values
pulsedmasterlogic.set_ext_microwave_settings(power=mw_power, frequency=mw_freq-100.0e+6, use_ext_microwave=True)
time.sleep(0.5)

In [25]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
# generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params=OrderedDict()
generate_params['name'] = 'pODMR_rf'
generate_params['microwave_amplitude'] = 0.25
generate_params['freq_step'] = 0.2e+6
generate_params['num_of_points'] = 150
generate_params['rf_freq'] = 1.7*1e+6
generate_params['rf_amp'] = 0.25
generate_params['n'] = 10

# get the current settings with the following property:
for setting, value in pulsedmasterlogic.ext_microwave_settings.items():
    print('{0}: {1}'.format(setting, value))

# Call the actual generation method using a wrapper method and pass the parameter set
pulsedmasterlogic.generate_predefined_sequence('pODMR_rf', generate_params)
# Give it a moment to generate
time.sleep(0.5)

# The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
# You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
#for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
#    print(ensemble_name)

# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_ensemble('pODMR_rf', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)
    
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= mw_freq + 
                                           np.arange(generate_params['num_of_points'])*generate_params['freq_step'],
                                           number_of_lasers=generate_params['num_of_points'], 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('Hz', 'arb. u.'))
time.sleep(0.5)

pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=generate_params['num_of_points'])
time.sleep(0.5)

# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))
import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))

power: -32.0
frequency: 1502000000.0
use_ext_microwave: True
{1: 'pODMR_rf_ch1', 2: 'pODMR_rf_ch2'}


In [7]:
poi=poimanager.active_poi.get_key()
NV_name =poimanager.poi_list[poi]._name
refocus_poi(poi)

131271.83333333334 4.401033545422124e-08


True

In [7]:
measurement_time = 180*60
refocus_interval = 2*60

In [8]:
pulsedmasterlogic.set_timer_interval(3)

In [ ]:
do_xy16_refocus(poi, measurement_time, refocus_interval)

In [24]:
pulsedmasterlogic.save_measurement_data(tag='pODMR_'+str(generate_params['rf_amp'])+str(generate_params['rf_freq']), with_error=True)

Measure matrix plot 

In [8]:
freq = 0.1*1e+6 + np.arange(40)*0.1*1e+6

for f in freq:
    # Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
    # generate_params = pulsedmasterlogic.generate_method_params['rabi']
    generate_params=OrderedDict()
    generate_params['name'] = 'pODMR_rf'
    generate_params['microwave_amplitude'] = 0.25
    generate_params['freq_step'] = 0.24e+6
    generate_params['num_of_points'] = 100
    generate_params['rf_freq'] = f
    generate_params['rf_amp'] = 0.25
    generate_params['n'] = 10

    # get the current settings with the following property:
    for setting, value in pulsedmasterlogic.ext_microwave_settings.items():
        print('{0}: {1}'.format(setting, value))

    # Call the actual generation method using a wrapper method and pass the parameter set
    pulsedmasterlogic.generate_predefined_sequence('pODMR_rf', generate_params)
    # Give it a moment to generate
    time.sleep(0.5)

    # The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
    # You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
    #for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
    #    print(ensemble_name)

    # The "with_load" flag can be used to also load the waveform into the channels after creating it
    pulsedmasterlogic.sample_ensemble('pODMR_rf', with_load=True)
    # Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
    while pulsedmasterlogic.status_dict['sampload_busy']:
        time.sleep(0.1)

    pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                               controlled_variable= mw_freq + 
                                               np.arange(generate_params['num_of_points'])*generate_params['freq_step'],
                                               number_of_lasers=generate_params['num_of_points'], 
                                               laser_ignore_list=[], 
                                               alternating=False, 
                                               units=('Hz', 'arb. u.'))
    time.sleep(0.5)

    pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=generate_params['num_of_points'])
    time.sleep(0.5)

    # You can get a read-only view of the current measurement settings by using this property:
    # for setting, value in pulsedmasterlogic.measurement_settings.items():
    #     print('{0}:\n  {1}'.format(setting, value))
    import winsound
    frequency = 300  # Set Frequency To 2500 Hertz
    duration = 1000  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    # You can get a read-only view of the current measurement settings by using this property:
    # for setting, value in pulsedmasterlogic.measurement_settings.items():
    #     print('{0}:\n  {1}'.format(setting, value))
    
    poi=poimanager.active_poi.get_key()
    NV_name =poimanager.poi_list[poi]._name
    refocus_poi(poi)
    time.sleep(3)
    
    measurement_time = 20*60
    refocus_interval = 2*60
    
    do_xy16_refocus(poi, measurement_time, refocus_interval)
    
    pulsedmasterlogic.save_measurement_data(tag='pODMR_'+str(generate_params['rf_amp'])+'_'+str(generate_params['rf_freq']), with_error=True)
    

power: -32.0
frequency: 1500000000.0
use_ext_microwave: True
{1: 'pODMR_rf_ch1', 2: 'pODMR_rf_ch2'}
131271.83333333334 5.628498445061004e-08
131271.83333333334 9.521279852121368e-09
131271.83333333334 3.2270637539697834e-08
131271.83333333334 2.5593188627924665e-08
131271.83333333334 1.0450964243658074e-08
131271.83333333334 8.78234791683465e-09
131271.83333333334 1.2888214976645225e-08
131271.83333333334 1.6801561321557043e-10
131271.83333333334 7.704961410029637e-09
131271.83333333334 1.0251633710872318e-08
power: -32.0
frequency: 1500000000.0
use_ext_microwave: True
{1: 'pODMR_rf_ch1', 2: 'pODMR_rf_ch2'}
131271.83333333334 2.7759667962038305e-08
131271.83333333334 5.066145338970888e-08
131271.83333333334 1.6069629893425354e-08
131271.83333333334 2.009801942722705e-10
131271.83333333334 3.2610992743082917e-09
131271.83333333334 5.074260982235049e-10
131271.83333333334 6.9021538321679065e-09
131271.83333333334 1.539686732317659e-09
131271.83333333334 6.177687859204063e-09
131271.83333